# IndoBenchmark: POSP

Named-entity recognition (NER) is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc. [[Wikipedia: Named Entity Recognition]](https://en.wikipedia.org/wiki/Named-entity_recognition)

We will try to finetune the bert-base-indonesian-522M for the Named Entity Recognition (NER) task. For this purpose we will use the [NERGRIT Corpus](https://github.com/grit-id/nergrit-corpus) which contains 321.757 lines of train, 66.974 lines of test and 64.208 lines of valid dataset. It uses Inside-Outside-Beginning (IOB) format where each line is composed of a word and its label/category. 

The [NERGRIT Corpus](https://github.com/grit-id/nergrit-corpus) is a very valueable dataset for indonesian NLP researcher. Unfortunately there are many typos or errors on the labels, so I spent some times to analyse the errors, make corrections and report the [issue to their Github repository](https://github.com/grit-id/nergrit-corpus/issues/1). Since the license allows us to redistribute the dataset, I will also publish the original dataset including its corrections. Currently the dataset is only available per [request](https://ner.grit.id/index.php/front/about) (klick the "Get NERGRIT Corpus").


## Transformers or Simpletransformers?

We will use simpletransformers in this case to simplify the training and inferencing

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
import logging
import sys

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

We use the corrected dataset which has less lines than the originals (train: 309203, valid: 61680, test: 64568)

In [3]:
data_dir = "/dataset/indonlu/posp_pos-prosa"
file_train = f'{data_dir}/train_preprocess.txt'
file_valid = f'{data_dir}/valid_preprocess.txt'
file_test = f'{data_dir}/test_preprocess_masked_label.txt'
#file_labels_map = f'{data_dir}/labels-map.csv'

The Simpletransformers requires the dataset either as pandas dataframe with following column/format: **sentence_id**, **words**, **labels**, or Text file in CoNLL format. The **sentence_id** is consecutive number determines which words belong to a given sentence. I.e. the words from the same sequence should be assigned the same unique sentence_id.

In [4]:
# Function to read ner file in connl format and return a DataFrame with columns: sentence_id, words, labels
def get_pos_data(filename, labels_map=None):
    word_list = []
    sentence_counter = 0
    with open(filename) as fp:
        for cnt, line in enumerate(fp):
            try:
                texts = line.split()
                if len(texts) != 0:
                    word, label = ' '.join(texts[0:-1]), texts[-1]
                    if labels_map:
                        label = labels_map[label]
                    word_list.append([sentence_counter, word, label])
                else:
                    sentence_counter += 1
            except:
                print("Unexpected error:", sys.exc_info()[0], cnt, line)
                word_list.append([sentence_counter, "", ""])              
                sentence_counter += 1
                pass
    print(f'read {cnt} lines')
    ner_data = pd.DataFrame(word_list, columns=["sentence_id", "words", "labels"])
    return ner_data

In [5]:
train_data = get_pos_data(file_train)

read 163128 lines


In [6]:
valid_data = get_pos_data(file_valid)

read 20468 lines


In [7]:
test_data = get_pos_data(file_test)

read 20791 lines


In [8]:
len(train_data),len(valid_data),len(test_data)

(156409, 19629, 19952)

In [9]:
labels = list(set(train_data['labels']))

In [10]:
labels.sort()

In [11]:
labels

['B-$$$',
 'B-ADJ',
 'B-ADK',
 'B-ADV',
 'B-ART',
 'B-CCN',
 'B-CSN',
 'B-INT',
 'B-KUA',
 'B-NEG',
 'B-NNO',
 'B-NNP',
 'B-NUM',
 'B-PAR',
 'B-PPO',
 'B-PRI',
 'B-PRK',
 'B-PRN',
 'B-PRR',
 'B-SYM',
 'B-UNS',
 'B-VBE',
 'B-VBI',
 'B-VBL',
 'B-VBP',
 'B-VBT']

In [12]:
train_data.iloc[:50]

,sentence_id,words,labels
0,0,kepala,B-NNO
1,0,dinas,B-VBP
2,0,tata,B-NNO
3,0,kota,B-NNO
4,0,manado,B-NNP
5,0,amos,B-NNP
6,0,kenda,B-NNP
7,0,menyatakan,B-VBT
8,0,tidak,B-NEG
9,0,tahu,B-VBI


In [13]:
valid_data.head(10)

,sentence_id,words,labels
0,0,"""",B-SYM
1,0,demi,B-PPO
2,0,kebersamaan,B-NNO
3,0,",",B-SYM
4,0,karena,B-CSN
5,0,menyangkut,B-VBT
6,0,kepentingan,B-NNO
7,0,parpol,B-NNO
8,0,",",B-SYM
9,0,maka,B-CSN


In [14]:
test_data.head(10)

,sentence_id,words,labels
0,0,kuasa,B-NNP
1,0,hukum,B-NNP
2,0,teamster,B-NNP
3,0,berasal,B-NNP
4,0,dari,B-NNP
5,0,edmonton,B-NNP
6,0,",",B-NNP
7,0,namun,B-NNP
8,0,tinggal,B-NNP
9,0,di,B-NNP


## The Labels

The NERGRIT corpus contains 19 entities, each with Inside- and Beginning-Tag, plus an Outside-Tag. Alltogether become 39 categories. The entities have following meaning:
1. 'CRD' --> Cardinal
1. 'DAT' --> Date
1. 'EVT' --> Event
1. 'FAC' --> Facility
1. 'GPE' --> Geopolitical Entity
1. 'LAW' --> Law Entity (such as Undang-Undang)
1. 'LOC' --> Location
1. 'MON' --> Money
1. 'NOR' --> Political Organization
1. 'ORD' --> Ordinal
1. 'ORG' --> Organization
1. 'PER' --> Person
1. 'PRC' --> Percent
1. 'PRD' --> Product
1. 'QTY' --> Quantity
1. 'REG' --> Religion
1. 'TIM' --> Time
1. 'WOA' --> Work of Art
1. 'LAN' --> Language

## The Training with bert-base-indonesian-522M

Since I have already pre-trained the bert-base with indonesian Wikipedia, I want to try its performance for this task.

In [42]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/posp/bert-base-indonesian-1.5G'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [43]:
model_args

NERArgs(adam_epsilon=1e-08, best_model_dir='/output/indonlu/posp/bert-base-indonesian-1.5G/best_model', cache_dir='cache_dir/', config={}, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=78, do_lower_case=True, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=True, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, model_name=None, model_type=None, multiprocessing_chunksize=500, n_gpu=1, no_cache=False, no_save=False, num_train_epochs=5, output_dir='/output/indonlu/posp/bert-base-indonesian-1.5G', overwrite_output_dir=True, 

In [44]:
model_bert_base = NERModel(
    #"bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
    "bert", "cahya/bert-base-indonesian-1.5G", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [45]:
# Train the model
model_bert_base.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/posp/bert-base-indonesian-1.5G.


In [46]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.14931659584066698, 'precision': 0.9636252483570228, 'recall': 0.9636252483570228, 'f1_score': 0.9636252483570227}


In [47]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 7.15584503809611, 'precision': 0.17762630312750602, 'recall': 0.17762630312750602, 'f1_score': 0.17762630312750602}


In [48]:
test_data.iloc[0:30]

,sentence_id,words,labels
0,0,kuasa,B-NNP
1,0,hukum,B-NNP
2,0,teamster,B-NNP
3,0,berasal,B-NNP
4,0,dari,B-NNP
5,0,edmonton,B-NNP
6,0,",",B-NNP
7,0,namun,B-NNP
8,0,tinggal,B-NNP
9,0,di,B-NNP


In [32]:
preds_list[:5]

[['B-NNO',
  'B-NNO',
  'B-NNP',
  'B-VBI',
  'B-PPO',
  'B-NNP',
  'B-SYM',
  'B-CCN',
  'B-VBI',
  'B-PPO',
  'B-PPO',
  'B-NNP',
  'B-PPO',
  'B-NNO',
  'B-NUM',
  'B-SYM'],
 ['B-NNO',
  'B-VBP',
  'B-PPO',
  'B-NNO',
  'B-SYM',
  'B-NNO',
  'B-NNO',
  'B-CCN',
  'B-NNO',
  'B-SYM',
  'B-NNO',
  'B-PRR',
  'B-VBP',
  'B-VBP',
  'B-PPO',
  'B-PPO',
  'B-NNO',
  'B-NNP',
  'B-VBI',
  'B-PPO',
  'B-NUM',
  'B-PPO',
  'B-NUM',
  'B-NNO',
  'B-PPO',
  'B-NNO',
  'B-NNO',
  'B-VBT',
  'B-NNO',
  'B-SYM'],
 ['B-SYM',
  'B-NNO',
  'B-PPO',
  'B-NNO',
  'B-VBI',
  'B-PPO',
  'B-NNO',
  'B-NNO',
  'B-NNP',
  'B-NNO',
  'B-CCN',
  'B-NNO',
  'B-SYM',
  'B-NNO',
  'B-PPO',
  'B-NNO',
  'B-VBT',
  'B-NNO',
  'B-PPO',
  'B-VBI',
  'B-NNO',
  'B-SYM',
  'B-VBT',
  'B-NNP',
  'B-SYM',
  'B-NNO',
  'B-VBP',
  'B-NNO',
  'B-SYM',
  'B-NNP',
  'B-SYM',
  'B-NUM',
  'B-SYM'],
 ['B-SYM',
  'B-PRN',
  'B-NNO',
  'B-ART',
  'B-NNO',
  'B-NNO',
  'B-SYM',
  'B-PPO',
  'B-NNO',
  'B-NNO',
  'B-ART',
  'B-PA

In [49]:
len(preds_list)

840

In [30]:
def print_result(preds_list, test_data, max_len=10):
    for i in range(len(preds_list)):
        if i>max_len:
            break
        sentence = list(test_data[test_data['sentence_id']==i]['words'])
        for j, word in enumerate(sentence):
            print(f'{i}:{word}\t{preds_list[i][j]}')

def save_result(preds_list, test_data, filename):
    with open(filename, 'w') as out_file:
        out_file.write(f'index,label\n')
        index = 0
        for i in range(len(preds_list)):
            label = str(preds_list[i])
            out_file.write(f'{index},"{label}"\n')
            index += 1


In [51]:
#output_dir = "/output/indonlu/nerp"
output_fn = f'{model_args.output_dir}/result.txt'

In [52]:
output_fn

'/output/indonlu/posp/bert-base-indonesian-1.5G/result.txt'

In [53]:
print_result(preds_list, test_data, 6)

0:kuasa	B-NNO
0:hukum	B-NNO
0:teamster	B-NNP
0:berasal	B-VBI
0:dari	B-PPO
0:edmonton	B-NNP
0:,	B-SYM
0:namun	B-CCN
0:tinggal	B-VBI
0:di	B-PPO
0:sekitar	B-PPO
0:vancouver	B-NNP
0:sejak	B-PPO
0:tahun	B-NNO
0:1991	B-NUM
0:.	B-SYM
1:data	B-NNO
1:diurutkan	B-VBP
1:berdasarkan	B-PPO
1:umur	B-NNO
1:,	B-SYM
1:jenis	B-NNO
1:kelamin	B-NNO
1:dan	B-CCN
1:metode	B-NNO
1:;	B-SYM
1:pertambahan	B-NNO
1:yang	B-PRR
1:ditandai	B-VBP
1:terlihat	B-VBP
1:di	B-PPO
1:antara	B-PPO
1:orang	B-NNO
1:skotlandia	B-NNP
1:berumur	B-VBI
1:dari	B-PPO
1:25	B-NUM
1:sampai	B-CSN
1:54	B-NUM
1:tahun	B-NNO
1:dengan	B-PPO
1:gantung	B-NNO
1:diri	B-NNO
1:meningkatkan	B-VBT
1:popularitas	B-NNO
1:.	B-SYM
2:"	B-SYM
2:urusan	B-NNO
2:dengan	B-PPO
2:atasannya	B-NNO
2:masuk	B-VBI
2:dalam	B-PPO
2:aturan	B-NNO
2:pelanggaran	B-NNO
2:uu	B-NNP
2:kepegawaian	B-NNP
2:dan	B-CCN
2:etika	B-NNO
2:.	B-SYM
2:presiden	B-NNO
2:sebagai	B-PPO
2:atasan	B-NNO
2:punya	B-VBT
2:hak	B-NNO
2:untuk	B-PPO
2:beri	B-VBI
2:sanksi,	B-NNO
2:"	B-SYM
2:kata	B-VBT
2:y

In [54]:
save_result(preds_list, test_data, output_fn)

In [55]:
!head $output_fn

index,label
0,"['B-NNO', 'B-NNO', 'B-NNP', 'B-VBI', 'B-PPO', 'B-NNP', 'B-SYM', 'B-CCN', 'B-VBI', 'B-PPO', 'B-PPO', 'B-NNP', 'B-PPO', 'B-NNO', 'B-NUM', 'B-SYM']"
1,"['B-NNO', 'B-VBP', 'B-PPO', 'B-NNO', 'B-SYM', 'B-NNO', 'B-NNO', 'B-CCN', 'B-NNO', 'B-SYM', 'B-NNO', 'B-PRR', 'B-VBP', 'B-VBP', 'B-PPO', 'B-PPO', 'B-NNO', 'B-NNP', 'B-VBI', 'B-PPO', 'B-NUM', 'B-CSN', 'B-NUM', 'B-NNO', 'B-PPO', 'B-NNO', 'B-NNO', 'B-VBT', 'B-NNO', 'B-SYM']"
2,"['B-SYM', 'B-NNO', 'B-PPO', 'B-NNO', 'B-VBI', 'B-PPO', 'B-NNO', 'B-NNO', 'B-NNP', 'B-NNP', 'B-CCN', 'B-NNO', 'B-SYM', 'B-NNO', 'B-PPO', 'B-NNO', 'B-VBT', 'B-NNO', 'B-PPO', 'B-VBI', 'B-NNO', 'B-SYM', 'B-VBT', 'B-NNP', 'B-SYM', 'B-NNO', 'B-VBP', 'B-NNO', 'B-SYM', 'B-NNP', 'B-SYM', 'B-NUM', 'B-SYM']"
3,"['B-SYM', 'B-PRN', 'B-NNO', 'B-ART', 'B-NNO', 'B-NNO', 'B-SYM', 'B-PPO', 'B-NNO', 'B-NNO', 'B-ART', 'B-PAR', 'B-NNO', 'B-ADJ', 'B-VBI', 'B-PRI', 'B-ADV', 'B-SYM', 'B-CCN', 'B-PRN', 'B-PPO', 'B-NNO', 'B-ADJ', 'B-SYM', 'B-NNO', 'B-PPO', 'B-NNO', 'B-SYM', 'B-NNO

In [56]:
result = pd.read_csv(output_fn, names=['index','label']).set_index('index')

In [41]:
result.iloc[0:20]

,label
index,
index,label
0,"['B-NNO', 'B-NNO', 'B-NNP', 'B-VBI', 'B-PPO', ..."
1,"['B-NNO', 'B-VBP', 'B-PPO', 'B-NNO', 'B-SYM', ..."
2,"['B-SYM', 'B-NNO', 'B-PPO', 'B-NNO', 'B-VBI', ..."
3,"['B-SYM', 'B-PRN', 'B-NNO', 'B-ART', 'B-NNO', ..."
4,"['B-KUA', 'B-ADJ', 'B-NNO', 'B-NNO', 'B-ADV', ..."
5,"['B-PPO', 'B-VBT', 'B-ADV', 'B-ADJ', 'B-SYM', ..."
6,"['B-VBI', 'B-ART', 'B-NNO', 'B-PRR', 'B-ADV', ..."
7,"['B-NNP', 'B-SYM', 'B-NNP', 'B-ADK', 'B-VBL', ..."


### Nergrit 2 (train+valid)

In [15]:
train_data_all = pd.concat([train_data, valid_data], ignore_index=True)

In [16]:
len(train_data_all), len(train_data), len(valid_data), 

(176038, 156409, 19629)

In [25]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/posp/bert-base-indonesian-1.5G-all'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [26]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-1.5G", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [27]:
# Train the model
model_bert_base.train_model(train_data_all, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/posp/bert-base-indonesian-1.5G-all.


In [28]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.030036924635281875, 'precision': 0.9913393448469102, 'recall': 0.9913393448469102, 'f1_score': 0.9913393448469102}


In [29]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 8.007513436816987, 'precision': 0.17647353648757017, 'recall': 0.17647353648757017, 'f1_score': 0.17647353648757017}


In [31]:
output_fn = f'{model_args.output_dir}/result.txt'

In [32]:
save_result(preds_list, test_data, output_fn)

In [33]:
output_fn

'/output/indonlu/posp/bert-base-indonesian-1.5G-all/result.txt'

In [28]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/bert-base-indonesian'
model_args.best_model_dir = '/output/indonlu/bert-base-indonesian/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [31]:
model_bert_base = NERModel(
    "bert", "/output/indonlu/bert-base-indonesian/best_model",  args=model_args
)

In [32]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.12883393300904167, 'precision': 0.7910662824207493, 'recall': 0.8293051359516617, 'f1_score': 0.8097345132743363}


In [33]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.7154410238619204, 'precision': 0.0, 'recall': 0, 'f1_score': 0}


In [34]:
save_result(preds_list, test_data_2, output_fn)

### Nergrit 1 vs Nergrit 2

In [41]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/bert-base-indonesian'
model_args.best_model_dir = '/output/indonlu/bert-base-indonesian/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [42]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [43]:
# Train the model
model_bert_base.train_model(train_data_1, eval_data=valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/bert-base-indonesian.


In [44]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.3372342694136832, 'precision': 0.6912378303198887, 'recall': 0.7507552870090635, 'f1_score': 0.7197682838522811}


In [39]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.3447435959069817, 'precision': 0.6963276836158192, 'recall': 0.7447129909365559, 'f1_score': 0.7197080291970803}


In [ ]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

In [ ]:
train_data_3 = pd.concat([train_data_1, train_data_2], ignore_index=True)

### (Nergrit 1 +  Nergrit 2) vs Nergrit 2

In [45]:
train_data_3 = pd.concat([train_data_1, train_data_2], ignore_index=True)

In [46]:
len(train_data_3),len(train_data_1),len(train_data_2)

(365416, 309206, 56210)

In [47]:
train_data_1.head()

,sentence_id,words,labels
0,0,Indonesia,B-PLACE
1,0,mengekspor,O
2,0,produk,O
3,0,industri,O
4,0,skala,O


In [48]:
last_si = train_data_1.iloc[-1]['sentence_id']

In [55]:
train_data_tmp = train_data_2

In [63]:
train_data_tmp['sentence_id'] = 100

In [80]:
train_data_tmp.head()

,sentence_id,words,labels
0,12552,Kontribusinya,O
1,12552,terhadap,O
2,12552,industri,O
3,12552,musik,O
4,12552,telah,O


In [67]:
train_data_2.head()

,sentence_id,words,labels
0,0,Kontribusinya,O
1,0,terhadap,O
2,0,industri,O
3,0,musik,O
4,0,telah,O


In [79]:
for i, row in train_data_2.iterrows():
    train_data_tmp.loc[i, 'sentence_id'] = train_data_2.iloc[i]['sentence_id'] + last_si + 1
    #print(i, train_data_tmp.iloc[i]['sentence_id'] )

In [81]:
train_data_3 = pd.concat([train_data_1, train_data_tmp], ignore_index=True)

In [82]:
train_data_3.iloc[309200: 309220]

,sentence_id,words,labels
309200,12551,di,O
309201,12551,38,O
309202,12551,negara,O
309203,12551,bagian,O
309204,12551,lainnya,O
309205,12551,.,O
309206,12552,Kontribusinya,O
309207,12552,terhadap,O
309208,12552,industri,O
309209,12552,musik,O


In [83]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [84]:
# Train the model
model_bert_base.train_model(train_data_3, eval_data=valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/bert-base-indonesian.


In [85]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.1967056412939672, 'precision': 0.7793904208998549, 'recall': 0.8111782477341389, 'f1_score': 0.7949666913397484}


In [24]:
preds_list

[['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'B-ORGANISATION', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PERSON',
  'I-PERSON',
  'I-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORGANISATION',
  'I-ORGANISATION',
  'O',
  'O',
  'O',
  'B-PLACE',
  'O',
  'B-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  

In [33]:
list(test_data[test_data['sentence_id']==0]['words'])

['Joetata',
 'Hadihardaja',
 'dan',
 'dihadiri',
 'oleh',
 'Rektor',
 'Undip',
 'Prof',
 '.']

In [32]:
for i in range(len(preds_list)):
    sentence = list(test_data[test_data['sentence_id']==i]['words'])
    for j, word in enumerate(sentence):
        print(word, preds_list[i][j])
    if i>10:
        break

NameError: name 'test_data' is not defined

In [28]:
for i, row in test_data.iterrows():
    print(i, row['words'], preds_list[row['sentence_id']])
    
    
    for j in row['words']:
    if i>10:
        break

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 1
10 1
11 1


The result (F1-score: 80.17 %) is quite similar with the F1-score NERGRIT has achieved (about 80%). 
Last week I got very low F1-score (about 60%), I was disappointed because it was much lower then the F1-score achieved by NERGRIT team. It turned out that the model was trained incorrectly, I trained the bert-base-indonesian-522M as if it is cased model (this is the default configuration). After I enabled the lowercase in the configuratin (model_args.do_lower_case = True), the F1-score is much better.


## The Training with xlm-roberta-base

I tried a multilanguage model from Facebook: XLM-Roberta-base which was pre-trained on 2.5TB of dataset.

In [19]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 128
model_args.evaluate_during_training = True
model_args.output_dir = '/output/ner/xlm-roberta-base'
model_args.best_model_dir = '/output/ner/xlm-roberta-base/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels

In [20]:
model_xlmroberta_base = NERModel(
    "xlmroberta", "xlm-roberta-base", labels=labels, args=model_args
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [21]:
# Train the model
model_xlmroberta_base.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of xlmroberta model complete. Saved to /output/ner/xlm-roberta-base.


In [22]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_xlmroberta_base.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.1866366565582298, 'precision': 0.8336475023562677, 'recall': 0.8462090408993064, 'f1_score': 0.8398813056379822}


In [23]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_xlmroberta_base.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.18713697187486106, 'precision': 0.8234437975817286, 'recall': 0.8321077044914583, 'f1_score': 0.8277530808620787}


### Result

The result is great, F1-score 82.8%

## The Training with xlm-roberta-large

Then I tried a second multilanguage model from Facebook: XLM-Roberta-large

In [24]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/ner/xlm-roberta-large'
model_args.best_model_dir = '/output/ner/xlm-roberta-large/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels

In [25]:
model_xlmroberta_large = NERModel(
    "xlmroberta", "xlm-roberta-large", labels=labels, args=model_args
)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

In [26]:
# Train the model
model_xlmroberta_large.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of xlmroberta model complete. Saved to /output/ner/xlm-roberta-large.


In [27]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_xlmroberta_large.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.21224965298888349, 'precision': 0.8447300165055411, 'recall': 0.8568524276488878, 'f1_score': 0.8507480408454049}


In [28]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_xlmroberta_large.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.21842414552079087, 'precision': 0.836591086786552, 'recall': 0.8473809254440547, 'f1_score': 0.8419514388489209}


### Result

Again, the result is great, it achieved F1-score of 84.19%. It is 4 percent better than the bert-base-indonesian-522M. Maybe  my LM needs more data for pre-training

## Predict some Samples

In [29]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)",
    "Selama 24 jam puncak Mahameru di Malang kebanjiran pendaki dari Wina",
]

In [30]:
predictions, raw_outputs = model_bert_base.predict(texts)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


[[{'Gubernur': 'B-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'B-ORG'},
  {'Gubernur': 'I-ORG'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'B-LOC'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-GPE'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-GPE'}]]

In [31]:
predictions, raw_outputs = model_xlmroberta_base.predict(texts)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


[[{'Gubernur': 'B-NOR'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'O'},
  {'Gubernur': 'I-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'B-LOC'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-GPE'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-GPE'}]]

In [32]:
predictions, raw_outputs = model_xlmroberta_large.predict(texts)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


[[{'Gubernur': 'B-ORG'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'O'},
  {'Gubernur': 'B-ORG'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'B-LOC'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-GPE'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-GPE'}]]